# Kısmi En küçük Kareler  Regresyonu

Partial least squares regression

* çok boyutluluk laneti
* çoklu doğrusal bağlantı problemi
* PLS de PCR gib ibağımsız değişkenlerin doğrusal kombinasyonlarını bulur
Bu doğrusal kombinasyonlar bileşen yada latent değişken olarak adlandırılır
* PLS NIPELS ın özel bir halidir, iteratif olarak bağımlı değişken ile
yüksek korolasyona sahip değişkenler arasındaki gizil(latent) ilişkiyi bulmaya çalışır 

* PCR da doğrusal kombinasyonlar yani bileşenşer *bağımsız değişken uzağındaki değişkneliği* 
maksimum şekilde özetleyecek şekilde oluşturulur
* Bu durum bağımlı değişkeni açıklma yeteneği olmamasına sebep olmakta
* PLS de ise *bileşenler bağımlı değişken ile olan kovaryansı* maksimumşekilde özetleyecek şekilde oşuşturulur
- yani hocam PCR değişkenlerde indirgeme yapar ancak bağımsız değişkenler arasında oluşturu bağımlı değişknele alakası yoktur
PLS ise bağımsız değişkenlerin bağımlı değişkene alakasına göre bir indirgeme yapar
- İşte bu yüzden değişknelre atılmak istenmiyorsa ve açıkklanabilirlik aranıyorsa PLS yoksa PCR kullanilir


# Model

In [1]:
import pandas as pd

In [3]:
hit=pd.read_csv("Hitters.csv")
df=hit.copy()
df=df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 263 entries, 1 to 321
Data columns (total 20 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   AtBat      263 non-null    int64  
 1   Hits       263 non-null    int64  
 2   HmRun      263 non-null    int64  
 3   Runs       263 non-null    int64  
 4   RBI        263 non-null    int64  
 5   Walks      263 non-null    int64  
 6   Years      263 non-null    int64  
 7   CAtBat     263 non-null    int64  
 8   CHits      263 non-null    int64  
 9   CHmRun     263 non-null    int64  
 10  CRuns      263 non-null    int64  
 11  CRBI       263 non-null    int64  
 12  CWalks     263 non-null    int64  
 13  League     263 non-null    object 
 14  Division   263 non-null    object 
 15  PutOuts    263 non-null    int64  
 16  Assists    263 non-null    int64  
 17  Errors     263 non-null    int64  
 18  Salary     263 non-null    float64
 19  NewLeague  263 non-null    object 
dtypes: float64

In [9]:
ms=pd.get_dummies(df[["League","Division","NewLeague"]]) #kategorik değişkeni numeric yapma one hot

In [17]:
y=df["Salary"]#bağımlı değişkenş alma

In [16]:
X_=df.drop(["League","Division","NewLeague"],axis=1).astype("float64") #kategoric değişkenleri sildik
X=pd.concat([X_, ms[["League_N","Division_W","NewLeague_N"]]],axis=1)#dönüşüm yapılmıi halini ekledik

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y,test_size=0.25,random_state=42) # eğitim ve test kısmını ayırdık

In [15]:
# buraya kadar PCR ile aynı bildiğin data yükleme temel preprocession

In [19]:
from sklearn.cross_decomposition import PLSRegression, PLSSVD

In [24]:
pls=PLSRegression(n_components=10) #hocam buradaki bileşen sayısını kaç yaparsan yap çıkna katsayı yine baştaki değişken sayısı ile
#aynı olur 20 değişken varsa sen burada kaç ayarlarsan ayarla 20 katsayı çıkar. zaten çıkmazsa sorun olıur modeli kurucaz ee diyecek
#değişken sayıları uymuyor. yani bileşen sayısnın ayarlanması direkt değişken sayısını değiştirmek değildir
pls=pls.fit(X_train, y_train)

In [25]:
pls.coef_

/Users/necmiyesoyla/Library/Python/3.9/lib/python/site-packages/sklearn/cross_decomposition/_pls.py:503: FutureWarning: The attribute `coef_` will be transposed in version 1.3 to be consistent with other linear models in scikit-learn. Currently, `coef_` has a shape of (n_features, n_targets) and in the future it will have a shape of (n_targets, n_features).
  warnings.warn(


array([[-9.14502020e+00],
       [ 1.04245638e+01],
       [ 1.86367804e+00],
       [-3.09078786e+00],
       [-1.25212555e+00],
       [ 2.04550727e+00],
       [ 1.03344435e+00],
       [-4.65331282e+00],
       [ 1.58926000e+00],
       [ 5.26500915e-01],
       [ 5.95529397e+00],
       [ 6.29274519e-01],
       [-4.83316481e+00],
       [ 8.38144495e-02],
       [ 2.35638890e-01],
       [ 7.95916033e-01],
       [ 4.44878316e+02],
       [-1.61809625e+00],
       [-2.95492381e-01],
       [ 1.69370299e+00]])